In [ ]:
!pip install colabcode


In [ ]:
from colabcode import  ColabCode
ColabCode(port=10000)


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# initialize the model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')


In [ ]:
# Intialize sentiment function to get sentiment probabilities for each tokes
def sentiment(tokens):
    # get output logits from the model
    output = model(**tokens)
    # convert to probabilities
    probs = torch.nn.functional.softmax(output[0], dim=-1)
    # return the probability tensor 
    return probs

In [ ]:
txt = """
Tokyo 2020: Two South African footballers test positive for Covid-19 in Olympic Village
CNN Expansion Hong Kong July 2020 795169, Jessie Yeung
By Jessie Yeung, CNN

Updated 1332 GMT (2132 HKT) July 18, 2021
(CNN)Just days before the Olympics are set to begin, three members from South Africa's Olympic football team have tested positive for Covid-19 after arriving at the Tokyo Olympic Village, according to the South African Football Association.

"We have three positive cases of COVID-19 in the camp here, two players and an official," team manager Mxolisi Sibam said in the press release Sunday.
The two athletes are footballers Thabiso Monyane and Kamohelo Mahlatsi and the official is video analyst Mario Masha, according to the association. They are the first athletes to have tested positive in the village.
The team left South Africa on July 13 and arrived in Tokyo the following day and had reported negative results.
The video analyst tested positive Thursday and went into isolation, while the athletes tested positive "on Friday and Saturday respectively," the association said.
The whole team is now under quarantine "until cleared to train," according to the the association said.
"We are now all in quarantine until we get cleared for training today, pending this morning's results. This unfortunate situation has made us miss our first intensive training session last night'' said Sibam.
The first case in the village was reported on Saturday after an individual, not believed to be an athlete, tested positive.
Outside the Olympic Village a third athlete tested positive on Sunday, said organizers. The names and nationalities of the positive cases were not made known by organizers.
The Tokyo 2020 Olympics will begin on Friday -- but concern is growing over the danger of Covid spreading, with 55 confirmed cases now linked to the Games, including officials and contractors. The Japanese public, as well as many international observers, have voiced alarm over the Games going forward as Japan struggles to rein in its latest coronavirus outbreak.
The country saw a huge second wave in the spring, peaking in April and May with close to 6,000 new cases per day. Cases began falling in June but have risen in recent weeks, sparking fears the arrival of teams from more than 200 countries could turn the Games into a global superspreader event.
As of Friday, more than 15,000 Olympic individuals had entered Japan, according to Thomas Bach, president of the International Olympic Committee (IOC). The Olympic Village, containing 21 residential buildings, will house about 11,000 athletes.
Bach acknowledged criticisms about staging the Olympics during the pandemic, but also defended the preventative measures that had been taken.
"We are well aware of the skepticism a number of people have here in Japan," he said at a news conference on Saturday. "My appeal to the Japanese people is to welcome the athletes for their competitions." Organizers announced this month that the Tokyo venues will not have spectators due to the city's coronavirus state of emergency -- an unprecedented move, according to an IOC spokesman.
The Olympic Village is prepped with Covid testing and health centers, with signs reminding residents to wear face masks and keep at least one meter (about 3.3 feet) away from each other. Athletes will be contact-traced and tested for Covid daily; if they test positive, they will be taken to an isolation facility outside the Olympic Village, and will not be able to compete.
Other measures in the village include limiting the capacity of indoor facilities, such as dining halls and training centers, where residents are separated by plastic barriers.
Still, some athletes have decided against the risk and pulled out of the games, including Australian tennis star Nick Kyrgios and Australian basketball player Liz Cambage. About 10,000 Olympic volunteers have also quit.
CNN's Selina Wang contributed to this report.
"""

In [ ]:
tokens = tokenizer.encode_plus(txt, add_special_tokens=False)
len(tokens['input_ids'])

In [ ]:
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

## Long text classification using windows

In [ ]:
probs_list = []
start = 0
window_size = 510

total_len = len(input_ids)

loop = True
while loop:
    end = start + window_size

    if end >= total_len:
        loop = False
        end = total_len
    
    # extract window from input_ids and attention_mask and add special token manually
    input_ids_chunk = [101] + input_ids[start:end] + [102]
    attention_mask_chunk = [1] + attention_mask[start:end] + [1]

    # Add padding
    input_ids_chunk += [0] * (window_size - len(input_ids_chunk) + 2)
    attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) + 2)

    print(len(input_ids_chunk), len(attention_mask_chunk))
    
    # Format two tensors into pytorch
    input_dict = {
        'input_ids': torch.Tensor([input_ids_chunk]).long(), # Long Tensor
        'attention_mask': torch.Tensor([attention_mask_chunk]).int(), # Integer tensor
    }

    probs_list.append(sentiment(input_dict))

    print(f'start={start}\nend={end}')
    start = end

In [ ]:
probs_list

In [ ]:
with torch.no_grad():
  stack = torch.stack(probs_list)
  stack
  stack=stack.resize_(stack.shape[0], stack.shape[2])
  avg_prob = stack.mean(dim=0)

In [ ]:
torch.argmax(avg_prob).item()